In [3]:
'''
This code creates and saves the results of using the SVM model across various model features
'''

##### RUN LOOP on Cost? ####
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from model_functions import *
from run_decision_tree import *
import datetime
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import metrics
file_location = r"C:\Users\edgil\Documents\Masters\dissertation\code64\data_set\CcyData.csv"
performance_store = {"data_size" : [], "Accuracy_Score" : [], "ntree": [],
                     "Info_Ratio" : [], "run_time" : [], "train_date_st": [], "test_date_st": []}
########################### Set Model Paramaters #############################
# this looks back over a set period as the memory for the LSTM
kernel = "rbf" 
cost = 5 
test_buffer = 5
data_size = 1500 # train every 3 months
# I.e. append the data into one df over each training window, but also use all available up until that point
concat_results = False 
# if the number is > 1, then the code takes that as the number of test points you want to use
test_split = 460 # roughly one month test ahead
# signal threshold, when using classifier
thold = 0.55
total_data_needed = get_total_data_needed(test_split,data_size,test_buffer)
# standardisation window
window = 17500
###### Set Targets ##############
trade_horizon = 24 # in hours
use_risk_adjusted = True # if True: training on the sharpe return else raw
use_binary = False # set to true if you are using the risk adjusted and want it binary for classification score
use_classifier = True
################### Standardise Entire Dataset using rolling lookback windows ###############
data_normed, model_features = initialise_process(file_location, trade_horizon, window, use_risk_adjusted)
#data_normed = data_normed.replace(np.nan, 0)
start_row = 5000
use_separated_chunk = False # Use a rolling window to train and test
trunc_data = data_normed.loc[:start_row,:]  
################ Loop through the full dataset in terms of the training and testing.
# standardise the data
#################### Set up training and testing ########################
print("start_row %s" % start_row)
#trunc_data.to_csv(r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\TruncData_%s.csv" % str(start_row)) 
# create data_set
train , test = create_train_test_file(trunc_data, data_size, test_split, test_buffer, concat_results)
train_sample = train[model_features]
test_sample = test[model_features]
# train the model
# verbose = 1 gives the output of the training.
print("Starting ML model now end date : %s" % train['Date'].iloc[-1] )
start_time = datetime.datetime.now()
results, acc_score = run_svm_model(train_sample, test_sample,use_classifier,
                                       use_risk_adjusted,kernel, cost)
run_time = datetime.datetime.now() - start_time
test_results = backtester(results, test, trade_horizon)[0]
strat_return = backtester(results, test, trade_horizon)[1]
information_ratio = backtester(results, test, trade_horizon)[2]
train_date = train['Date'].iloc[0]
test_date = test['Date'].iloc[0]
performance_df = update_performance(data_size, cost, acc_score , information_ratio, 
                                    run_time, train_date, test_date, performance_store)
save_test_df = r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\svm\testresults_df_st_row%s_cost%s_use_risk%s_use_SepChunk%s_concat%s.csv" % (start_row,cost,use_risk_adjusted,use_separated_chunk,concat_results) 
if concat_results:
    # i.e. on the first iteration, keep reulsts as is, then we appedn at subsequent iterations
    if start_row == data_size:
        master_df = test_results
    else:
        master_df = pd.merge(master_df, test_results, how= "left", on= "Date").fillna(0)
else:
    test_results.to_csv(save_test_df, index = False)
print("finished with row: %s" % str(start_row))
    
save_perf_df = r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\svm\performance_df_cost%s_st_row%s.csv" % (cost,start_row) 
performance_df.to_csv(save_perf_df,index = False)
if concat_results:
    master_df.to_csv(save_test_df, index = False)

C:\Users\edgil\Documents\Masters\dissertation\code\model_functions.py:77: RuntimeWarning: divide by zero encountered in true_divide
  return data_df['logret'].iloc[::-1].shift(2).rolling(trade_horizon).sum().values[::-1]/data_df['logret'].iloc[::-1].shift(2).rolling(trade_horizon).std().values[::-1]
C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


start_row 5000
Starting ML model now end date : 2003-01-14 15:00:00
finished with row: 5000


In [3]:
'''
This code creates and saves the results of using the SVM model across various model features
'''

##### RUN LOOP on Cost? ####
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from model_functions import *
from run_decision_tree import *
import datetime
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import metrics
file_location = r"C:\Users\edgil\Documents\Masters\dissertation\code64\data_set\CcyData.csv"
performance_store = {"data_size" : [], "Accuracy_Score" : [], "ntree": [],
                     "Info_Ratio" : [], "run_time" : [], "train_date_st": [], "test_date_st": []}
########################### Set Model Paramaters #############################
# this looks back over a set period as the memory for the LSTM
kernel = "rbf" 
costs = [5] 
test_buffer = 5
data_size = 500 # train every 3 months ~ 15000
# I.e. append the data into one df over each training window, but also use all available up until that point
concat_results = True 
# if the number is > 1, then the code takes that as the number of test points you want to use
test_split = 500 # roughly one month test ahead
# signal threshold, when using classifier
thold = 0.55
total_data_needed = get_total_data_needed(test_split,data_size,test_buffer)
# standardisation window
window = 17500
###### Set Targets ##############
trade_horizon = 24 # in hours
use_risk_adjusted = False # if True: training on the sharpe return else raw
use_binary = False # set to true if you are using the risk adjusted and want it binary for classification score
use_classifier = True
################### Standardise Entire Dataset using rolling lookback windows ###############
data_normed, model_features = initialise_process(file_location, trade_horizon, window, use_risk_adjusted)
#data_normed = data_normed.replace(np.nan, 0)
start_row = data_size
use_separated_chunk = False # Use a rolling window to train and test
while start_row < data_normed.shape[0]:
    # first check if there is enough data left
    if (start_row + total_data_needed) > data_normed.shape[0]:
        # if we are about to go over the limit, then just return the last data_size + test size proportion of data
        trunc_data = data_normed.iloc[-total_data_needed:,:]
    # we need to increment over the data size
    if use_separated_chunk:
        # this means we jump across the full previous train and test data
        trunc_data = data_normed.loc[start_row:,:]
        start_row += total_data_needed
    if concat_results:
        # in this instance, we can to add to the start row first before chunking the data
        start_row += test_split
        # we are training on all data available up until that point, and testing x timeperiods ahead
        trunc_data = data_normed.loc[:start_row,:]  
    else:
        # this rolls the data so that the new training will overlap on the old test set and create a new separated test set
        trunc_data = data_normed.loc[start_row:,:]
        start_row += data_size
    trunc_data.to_csv(r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\svm\datatrunc_st_row%s.csv" % start_row)   
    print("start_row %s" % start_row)
    #trunc_data.to_csv(r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\TruncData_%s.csv" % str(start_row)) 
    # create data_set
    train , test = create_train_test_file(trunc_data, data_size, test_split, test_buffer, concat_results)
    train_sample = train[model_features]
    test_sample = test[model_features]
    # train the model
    # verbose = 1 gives the output of the training.
    print("Starting ML model now end date : %s" % train['Date'].iloc[-1] )
    start_time = datetime.datetime.now()
    for cost in costs:
        results, acc_score = run_svm_model(train_sample, test_sample,use_classifier,
                                           use_risk_adjusted,kernel, cost)
        run_time = datetime.datetime.now() - start_time
        test_results = backtester(results, test, trade_horizon)[0]
        strat_return = backtester(results, test, trade_horizon)[1]
        information_ratio = backtester(results, test, trade_horizon)[2]
        train_date = train['Date'].iloc[0]
        test_date = test['Date'].iloc[0]
        performance_df = update_performance(data_size, cost, acc_score , information_ratio, 
                                        run_time, train_date, test_date, performance_store)
    save_test_df = r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\svm\testresults_df_st_row%s_cost%s_use_risk%s_use_SepChunk%s_concat%s.csv" % (start_row,cost,use_risk_adjusted,use_separated_chunk,concat_results) 
    if concat_results:
        # i.e. on the first iteration, keep reulsts as is, then we appedn at subsequent iterations
        if start_row == (data_size + test_split):
            master_df = test_results
        else:
            master_df = pd.concat([master_df, test_results], axis = 0).reset_index(drop = True)
    else:
        test_results.to_csv(save_test_df, index = False)
    print("finished with row: %s" % str(start_row))
    if start_row == 5000:
        break
    save_perf_df = r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\svm\performance_df_cost%s_st_row%s_userisk%s.csv" % (cost,start_row, use_risk_adjusted) 
    performance_df.to_csv(save_perf_df,index = False)
if concat_results:
    master_df.to_csv(save_test_df, index = False)

start_row 1000
Starting ML model now end date : 2002-11-15 19:00:00
finished with row: 1000
start_row 1500
Starting ML model now end date : 2002-12-16 15:00:00
finished with row: 1500


C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


start_row 2000
Starting ML model now end date : 2003-01-14 11:00:00
finished with row: 2000


C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


start_row 2500
Starting ML model now end date : 2003-02-12 07:00:00


...

C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


finished with row: 4500


start_row 5000
Starting ML model now end date : 2003-07-08 11:00:00


C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


finished with row: 5000


In [2]:
'''
This code creates and saves the results of using the SVM model across various model features
'''

##### RUN PCA on Cost? ####
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from model_functions import *
from run_decision_tree import *
import datetime
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import metrics
file_location = r"C:\Users\edgil\Documents\Masters\dissertation\code64\data_set\CcyData.csv"
performance_store = {"data_size" : [], "Accuracy_Score" : [], "ntree": [],
                     "Info_Ratio" : [], "run_time" : [], "train_date_st": [], "test_date_st": []}
########################### Set Model Paramaters #############################
# this looks back over a set period as the memory for the LSTM
kernel = svm_dict['kerel'] 
costs = svm_dict['cost']
test_buffer = 5
data_size = 15000 # train every 3 months ~ 15000
# I.e. append the data into one df over each training window, but also use all available up until that point
concat_results = False 
# if the number is > 1, then the code takes that as the number of test points you want to use
test_split = 0.25 # roughly one month test ahead
# signal threshold, when using classifier
thold = 0.55
total_data_needed = get_total_data_needed(test_split,data_size,test_buffer)
# standardisation window
window = 17500
###### Set Targets ##############
trade_horizon = 120 # in hours
use_risk_adjusted = False # if True: training on the sharpe return else raw
use_binary = False # set to true if you are using the risk adjusted and want it binary for classification score
use_classifier = True
use_pca = 0 # if = 0 then implies do not use pca in the model
################### Standardise Entire Dataset using rolling lookback windows ###############
data_normed, model_features, features_to_standardise = initialise_process(file_location, trade_horizon, 
                                                 window, use_risk_adjusted, use_pca)
#data_normed = data_normed.replace(np.nan, 0)
start_row = data_size
use_separated_chunk = False # Use a rolling window to train and test
while start_row < data_normed.shape[0]:
    # first check if there is enough data left
    if (start_row + total_data_needed) > data_normed.shape[0]:
        # if we are about to go over the limit, then just return the last data_size + test size proportion of data
        trunc_data = data_normed.iloc[-total_data_needed:,:]
    # we need to increment over the data size
    if use_separated_chunk:
        # this means we jump across the full previous train and test data
        trunc_data = data_normed.loc[start_row:,:]
        start_row += total_data_needed
    if concat_results:
        # in this instance, we can to add to the start row first before chunking the data
        start_row += test_split
        # we are training on all data available up until that point, and testing x timeperiods ahead
        trunc_data = data_normed.loc[:start_row,:]  
    else:
        # this rolls the data so that the new training will overlap on the old test set and create a new separated test set
        trunc_data = data_normed.loc[start_row:,:]
        start_row += data_size
    #trunc_data.to_csv(r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\svm\datatrunc_st_row%s.csv" % start_row)   
    print("start_row %s" % start_row)
    #trunc_data.to_csv(r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\TruncData_%s.csv" % str(start_row)) 
    # create data_set
    train , test = create_train_test_file(trunc_data, data_size, test_split, test_buffer, concat_results)
    if use_pca > 0:
        train, test, var_explained = get_pca_features(train, test, features_to_standardise, use_pca)
    train_sample = train[model_features]
    test_sample = test[model_features]
    # train the model
    # verbose = 1 gives the output of the training.
    print("Starting ML model now end date : %s" % train['Date'].iloc[-1] )
    start_time = datetime.datetime.now()
    for cost in costs:
        results, acc_score = run_svm_model(train_sample, test_sample,use_classifier,
                                           use_risk_adjusted,kernel, cost)
        run_time = datetime.datetime.now() - start_time
        test_results = backtester(results, test, trade_horizon)[0]
        strat_return = backtester(results, test, trade_horizon)[1]
        information_ratio = backtester(results, test, trade_horizon)[2]
        train_date = train['Date'].iloc[0]
        test_date = test['Date'].iloc[0]
        performance_df = update_performance(data_size, cost, acc_score , information_ratio, 
                                        run_time, train_date, test_date, performance_store)
    save_test_df = r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\svm\testresults_df_st_row%s_cost%s_use_risk%s_use_SepChunk%s_concat%s_tradeH%s.csv"  % (start_row,cost,use_risk_adjusted,use_separated_chunk,concat_results,trade_horizon) 
    if concat_results:
        # i.e. on the first iteration, keep reulsts as is, then we appedn at subsequent iterations
        if start_row == (data_size + test_split):
            master_df = test_results
        else:
            master_df = pd.concat([master_df, test_results], axis = 0).reset_index(drop = True)
    else:
        test_results.to_csv(save_test_df, index = False)
    print("finished with row: %s" % str(start_row))

save_perf_df = r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\svm\performance_df_cost%s_st_row%s_userisk%s.csv" % (cost,start_row, use_risk_adjusted) 
performance_df.to_csv(save_perf_df,index = False)
if concat_results:
    master_df.to_csv(save_test_df, index = False)

start_row 30000
Starting ML model now end date : 2007-08-03 03:00:00


C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


finished with row: 30000
start_row 45000
Starting ML model now end date : 2009-12-25 03:00:00


C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


finished with row: 45000
start_row 60000
Starting ML model now end date : 2012-05-18 03:00:00


...

finished with row: 75000
start_row 90000
Starting ML model now end date : 2017-03-03 03:00:00


C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


finished with row: 90000
start_row 105000
Starting ML model now end date : 2017-11-30 09:00:00


C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


finished with row: 105000


In [1]:
##### FOR PCA features  && train on random data!####
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from model_functions import *
from run_decision_tree import *
import datetime
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import metrics

file_location = r"C:\Users\edgil\Documents\Masters\dissertation\code64\data_set\ccyDataBLIND_STFEATURES.csv"
performance_store = {"data_size" : [], "Accuracy_Score" : [], "ntree": [],
                     "Info_Ratio" : [], "run_time" : [], "train_date_st": [], "test_date_st": []}
params_dict = set_params_random_forests()
svm_dict = set_params_svm()
use_random_train_data = params_dict['use_random_train_data']
########################### Set Model Paramaters #############################
# this looks back over a set period as the memory for the LSTM
kernel = svm_dict['kernel'] 
costs = svm_dict['cost']# [i for i in range(25,301,25)] # [21, 66]
# if running pca, max features can only be same or less than the full total of features
test_buffer = params_dict['test_buffer']
data_size = params_dict['data_size'] #  initially using 1500 training points
# I.e. append the data into one df over each training window, but also use all available up until that point
concat_results = params_dict['concat_results']
# if the number is > 1, then the code takes that as the number of test points you want to use
test_split = params_dict['test_split'] # roughly one month test ahead, which is a one month retrain period
# signal threshold, when using classifier
thold = params_dict['thold']
total_data_needed = get_total_data_needed(test_split,data_size,test_buffer)
# standardisation window
window = params_dict['window']
###### Set Targets ##############
trade_horizon = params_dict['trade_horizon'] # in hours
use_risk_adjusted = params_dict['use_risk_adjusted'] # if True: training on the sharpe return else raw
use_binary = params_dict['use_binary'] # set to true if you are using the risk adjusted and want it binary for classification score
use_classifier = params_dict['use_classifier']
use_pca = params_dict['use_pca'] # if = 0 then implies do not use pca in the model
use_separated_chunk = params_dict['use_separated_chunk']
################### Standardise Entire Dataset using rolling lookback windows ###############
data_normed, model_features, features_to_standardise = initialise_process(file_location, trade_horizon, 
                                                 window, use_risk_adjusted, use_pca,use_random_train_data)
#data_normed = data_normed.replace(np.nan, 0)
start_row = data_size
 # Use a rolling window to train and test
################ Loop through the full dataset in terms of the training and testing.
if use_random_train_data:
    random_data_location = r"C:\Users\edgil\Documents\Masters\dissertation\code64\data_set\CcyRandomTrend.csv"
    train, model_features, features_to_standardise = initialise_process(random_data_location, trade_horizon, 
                                                 window, use_risk_adjusted, use_pca, use_random_train_data)
    test, model_features, features_to_standardise  = initialise_process(file_location, trade_horizon, 
                                                 window, use_risk_adjusted, use_pca, 
                                                use_random_train_data= False)
    if use_pca > 0:
        train, test, var_explained = get_pca_features(train,test, features_to_standardise, use_pca)
    train_sample = train[model_features]
    test_sample = test[model_features]
    for cost in costs:
        results, acc_score = run_svm_model(train_sample, test_sample,use_classifier, use_risk_adjusted,kernel,cost)
        test_results = backtester(results, test, trade_horizon)[0]
        strat_return = backtester(results, test, trade_horizon)[1]
        information_ratio = backtester(results, test, trade_horizon)[2]
        train_date = 1 # train['Date'].iloc[0]
        test_date = test['Date'].iloc[0]
        run_time = "random"
        performance_df = update_performance(data_size, cost, acc_score , information_ratio, 
                                            run_time, train_date, test_date, performance_store)
    save_test_df = r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\svm\randomData%s_COST%s_use_risk%s_use_SepChunk%s_concat%s_TH%s.csv" % (kernel,
                                    cost,use_risk_adjusted,use_separated_chunk,concat_results, trade_horizon) 
    test_results.to_csv(save_test_df, index = False)
else:
    # if not using random data then move to the normal method.
    while start_row < data_normed.shape[0]:
        # first check if there is enough data left
        if (start_row + total_data_needed) > data_normed.shape[0]:
            # if we are about to go over the limit, then just return the last data_size + test size proportion of data
            trunc_data = data_normed.iloc[-total_data_needed:,:]
        # we need to increment over the data size
        if use_separated_chunk:
            # this means we jump across the full previous train and test data
            trunc_data = data_normed.loc[start_row:,:]
            start_row += total_data_needed
        if concat_results:
            # in this instance, we can to add to the start row first before chunking the data
            start_row += test_split
            # we are training on all data available up until that point, and testing x timeperiods ahead
            trunc_data = data_normed.loc[:start_row,:]  
        else:
            # this rolls the data so that the new training will overlap on the old test set and create a new separated test set
            trunc_data = data_normed.loc[start_row:,:]
            start_row += data_size
        #trunc_data.to_csv(r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\svm\datatrunc_st_row%s.csv" % start_row)   
        print("start_row %s" % start_row)
        #trunc_data.to_csv(r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\TruncData_%s.csv" % str(start_row)) 
        # create data_set
        train , test = create_train_test_file(trunc_data, data_size, test_split, test_buffer, concat_results)
        if use_pca > 0:
            train, test, var_explained = get_pca_features(train, test, features_to_standardise, use_pca)
        train_sample = train[model_features]
        test_sample = test[model_features]
        # train the model
        # verbose = 1 gives the output of the training.
        print("Starting ML model now end date : %s" % train['Date'].iloc[-1] )
        start_time = datetime.datetime.now()
        for cost in costs:
            results, acc_score = run_svm_model(train_sample, test_sample,use_classifier,
                                               use_risk_adjusted,kernel, cost)
            run_time = datetime.datetime.now() - start_time
            test_results = backtester(results, test, trade_horizon)[0]
            strat_return = backtester(results, test, trade_horizon)[1]
            information_ratio = backtester(results, test, trade_horizon)[2]
            train_date = train['Date'].iloc[0]
            test_date = test['Date'].iloc[0]
            performance_df = update_performance(data_size, cost, acc_score , information_ratio, 
                                            run_time, train_date, test_date, performance_store)
        save_test_df = r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\svm\testresults_df_st_row%s_cost%s_use_risk%s_use_SepChunk%s_concat%s_tradeH%s.csv"  % (start_row,cost,use_risk_adjusted,use_separated_chunk,concat_results,trade_horizon) 
        if concat_results:
            # i.e. on the first iteration, keep reulsts as is, then we appedn at subsequent iterations
            if start_row == (data_size + test_split):
                master_df = test_results
            else:
                master_df = pd.concat([master_df, test_results], axis = 0).reset_index(drop = True)
        else:
            test_results.to_csv(save_test_df, index = False)
        print("finished with row: %s" % str(start_row))
    
    save_perf_df = r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\svm\performance_df_cost%s_st_row%s_userisk%s.csv" % (cost,start_row, use_risk_adjusted) 
    performance_df.to_csv(save_perf_df,index = False)
    if concat_results:
        master_df.to_csv(save_test_df, index = False)

C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
